In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import SMOTE
from math import log

In [2]:
dates = ['02', '03', '04', '05']
Data = pd.read_csv("C:/Users/India/Documents/Project2k23/Hard Disk failure/dataset/2019-07-01.csv")
for i in dates:
  Data = Data.append(pd.read_csv("C:/Users/India/Documents/Project2k23/Hard Disk failure/dataset/2019-07-" + i + ".csv"))

C:\Users\India\AppData\Local\Temp\ipykernel_19308\1840437860.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Data = Data.append(pd.read_csv("C:/Users/India/Documents/Project2k23/Hard Disk failure/dataset/2019-07-" + i + ".csv"))
C:\Users\India\AppData\Local\Temp\ipykernel_19308\1840437860.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Data = Data.append(pd.read_csv("C:/Users/India/Documents/Project2k23/Hard Disk failure/dataset/2019-07-" + i + ".csv"))
C:\Users\India\AppData\Local\Temp\ipykernel_19308\1840437860.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Data = Data.append(pd.read_csv("C:/Users/India/Documents/Project2k23/Hard Disk failure/dataset/2019-07-" + i + ".csv"))
C:\Users\India\AppData\Local\Temp\

In [3]:
Data.reset_index(inplace = True)
Data['date'] = pd.to_datetime(Data['date'])
Data.columns

Index(['index', 'date', 'serial_number', 'model', 'capacity_bytes', 'failure',
       'smart_1_normalized', 'smart_1_raw', 'smart_2_normalized',
       'smart_2_raw',
       ...
       'smart_250_normalized', 'smart_250_raw', 'smart_251_normalized',
       'smart_251_raw', 'smart_252_normalized', 'smart_252_raw',
       'smart_254_normalized', 'smart_254_raw', 'smart_255_normalized',
       'smart_255_raw'],
      dtype='object', length=130)

In [4]:
Data.drop('index', axis = 1, inplace = True)
Test = Data.dropna(how = 'all', axis = 1)
Initial = Data.columns
Initial

Index(['date', 'serial_number', 'model', 'capacity_bytes', 'failure',
       'smart_1_normalized', 'smart_1_raw', 'smart_2_normalized',
       'smart_2_raw', 'smart_3_normalized',
       ...
       'smart_250_normalized', 'smart_250_raw', 'smart_251_normalized',
       'smart_251_raw', 'smart_252_normalized', 'smart_252_raw',
       'smart_254_normalized', 'smart_254_raw', 'smart_255_normalized',
       'smart_255_raw'],
      dtype='object', length=129)

In [5]:
Features = ['5', '10', '184', '187', '188', '196', '197', '198']
len(Features)

8

In [6]:
for i in Features:
  features = ['smart_' + i + '_normalized', 'smart_' + i + '_raw']
  Test.drop(features, axis = 1, inplace = True)

C:\Users\India\AppData\Local\Temp\ipykernel_19308\2403248303.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test.drop(features, axis = 1, inplace = True)
C:\Users\India\AppData\Local\Temp\ipykernel_19308\2403248303.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test.drop(features, axis = 1, inplace = True)
C:\Users\India\AppData\Local\Temp\ipykernel_19308\2403248303.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test.dro

In [7]:
models = Test.model.value_counts().index
print("Number of Unique Models: ", len(models))
print(models)

Number of Unique Models:  41
Index(['ST12000NM0007', 'ST4000DM000', 'ST8000NM0055', 'HGST HMS5C4040BLE640',
       'ST8000DM002', 'HGST HUH721212ALN604', 'HGST HMS5C4040ALE640',
       'TOSHIBA MG07ACA14TA', 'ST10000NM0086', 'HGST HUH728080ALE600',
       'ST6000DX000', 'HGST HUH721212ALE600', 'ST500LM012 HN',
       'TOSHIBA MQ01ABF050', 'TOSHIBA MQ01ABF050M', 'ST500LM030',
       'WDC WD5000LPVX', 'TOSHIBA MD04ABA400V', 'WDC WD5000LPCX',
       'ST4000DM005', 'Seagate SSD', 'ST500LM021', 'HGST HUS726040ALE610',
       'HGST HDS5C4040ALE630', 'ST8000DM005', 'TOSHIBA HDWF180', 'DELLBOSS VD',
       'HGST HUH721010ALE600', 'Seagate BarraCuda SSD ZA500CM10002',
       'WDC WD5000BPKT', 'ST6000DM001', 'TOSHIBA HDWE160',
       'Seagate BarraCuda SSD ZA2000CM10002', 'WDC WD60EFRX', 'ST8000DM004',
       'Hitachi HDS5C4040ALE630', 'ST320LT007', 'ST9250315AS',
       'HGST HMS5C4040BLE641', 'ST1000LM024 HN', 'ST6000DM004'],
      dtype='object')


In [8]:
for i in models:
  try:
    c = Test[Test.model == i].failure.value_counts()[1] / len(Test[Test.model == i]) * 100
    print(i, 'failure_rate is {:.5f} %'.format(c))
  except KeyError:
    print(i, 'failure_rate is 0 %')

ST12000NM0007 failure_rate is 0.00922 %
ST4000DM000 failure_rate is 0.00204 %
ST8000NM0055 failure_rate is 0.00417 %
HGST HMS5C4040BLE640 failure_rate is 0 %
ST8000DM002 failure_rate is 0.00405 %
HGST HUH721212ALN604 failure_rate is 0 %
HGST HMS5C4040ALE640 failure_rate is 0 %
TOSHIBA MG07ACA14TA failure_rate is 0 %
ST10000NM0086 failure_rate is 0.01667 %
HGST HUH728080ALE600 failure_rate is 0 %
ST6000DX000 failure_rate is 0 %
HGST HUH721212ALE600 failure_rate is 0 %
ST500LM012 HN failure_rate is 0 %
TOSHIBA MQ01ABF050 failure_rate is 0 %
TOSHIBA MQ01ABF050M failure_rate is 0 %
ST500LM030 failure_rate is 0.08584 %
WDC WD5000LPVX failure_rate is 0 %
TOSHIBA MD04ABA400V failure_rate is 0 %
WDC WD5000LPCX failure_rate is 0 %
ST4000DM005 failure_rate is 0 %
Seagate SSD failure_rate is 0 %
ST500LM021 failure_rate is 0 %
HGST HUS726040ALE610 failure_rate is 0 %
HGST HDS5C4040ALE630 failure_rate is 0 %
ST8000DM005 failure_rate is 0 %
TOSHIBA HDWF180 failure_rate is 0 %
DELLBOSS VD failure_rat

In [9]:
M_features = Test.columns
print(Test.isnull().sum(axis = 0) / len(Test))
index = np.where(Test.isnull().sum(axis = 0) / len(Test) >= 0.5)[0]
len(index) 
M_features[index]

date                    0.000000
serial_number           0.000000
model                   0.000000
capacity_bytes          0.000000
failure                 0.000000
                          ...   
smart_251_raw           0.999991
smart_252_normalized    0.999991
smart_252_raw           0.999991
smart_254_normalized    0.997483
smart_254_raw           0.997483
Length: 99, dtype: float64


Index(['smart_2_normalized', 'smart_2_raw', 'smart_8_normalized',
       'smart_8_raw', 'smart_11_normalized', 'smart_11_raw',
       'smart_16_normalized', 'smart_16_raw', 'smart_17_normalized',
       'smart_17_raw', 'smart_22_normalized', 'smart_22_raw',
       'smart_23_normalized', 'smart_23_raw', 'smart_24_normalized',
       'smart_24_raw', 'smart_168_normalized', 'smart_168_raw',
       'smart_170_normalized', 'smart_170_raw', 'smart_173_normalized',
       'smart_173_raw', 'smart_174_normalized', 'smart_174_raw',
       'smart_177_normalized', 'smart_177_raw', 'smart_183_normalized',
       'smart_183_raw', 'smart_189_normalized', 'smart_189_raw',
       'smart_191_normalized', 'smart_191_raw', 'smart_200_normalized',
       'smart_200_raw', 'smart_218_normalized', 'smart_218_raw',
       'smart_220_normalized', 'smart_220_raw', 'smart_222_normalized',
       'smart_222_raw', 'smart_223_normalized', 'smart_223_raw',
       'smart_224_normalized', 'smart_224_raw', 'smart_225_no

In [10]:
f = ['date',	'serial_number',	'model',	'capacity_bytes',	'failure']

In [11]:
Null = np.where(Data.isnull().sum(axis = 0) == len(Data))[0]
len(Initial[Null])

14

In [12]:
Data.dropna(how = 'all', axis = 1, inplace = True) 
print(Data.shape)
del Test 

Data.drop(M_features[index], axis = 1, inplace = True)
Data.shape 

RE = Data.columns 
RE

(553380, 115)


Index(['date', 'serial_number', 'model', 'capacity_bytes', 'failure',
       'smart_1_normalized', 'smart_1_raw', 'smart_3_normalized',
       'smart_3_raw', 'smart_4_normalized', 'smart_4_raw',
       'smart_5_normalized', 'smart_5_raw', 'smart_7_normalized',
       'smart_7_raw', 'smart_9_normalized', 'smart_9_raw',
       'smart_10_normalized', 'smart_10_raw', 'smart_12_normalized',
       'smart_12_raw', 'smart_184_normalized', 'smart_184_raw',
       'smart_187_normalized', 'smart_187_raw', 'smart_188_normalized',
       'smart_188_raw', 'smart_190_normalized', 'smart_190_raw',
       'smart_192_normalized', 'smart_192_raw', 'smart_193_normalized',
       'smart_193_raw', 'smart_194_normalized', 'smart_194_raw',
       'smart_195_normalized', 'smart_195_raw', 'smart_196_normalized',
       'smart_196_raw', 'smart_197_normalized', 'smart_197_raw',
       'smart_198_normalized', 'smart_198_raw', 'smart_199_normalized',
       'smart_199_raw', 'smart_240_normalized', 'smart_240_raw',

In [13]:
Data['capacity_bytes'] = (Data['capacity_bytes'] // 1e+9)
Data['capacity_bytes'].value_counts() 
Data.drop(np.where(Data['capacity_bytes'] == -1)[0], inplace = True)
Data = Data.fillna(0)
Data.isnull().sum(axis = 0)
Test = Data.copy()
Test.shape 

(553328, 51)

In [14]:
def Sigmoid(x):
  return 1 / (1 + np.exp(-(x + np.random.normal(scale = 0.5)))) 
def TanH(x):
  return np.tanh(x + np.random.normal(scale = 0.5))

In [15]:
cf = ['smart_1_raw', 'smart_7_raw', 'smart_188_raw', 'smart_193_raw', 'smart_195_raw', 'smart_240_raw', 'smart_241_raw', 'smart_242_raw']
for i in cf:
   n = re.findall('\d+',i) 
   s = 'smart_' + ''.join(n) + '_sig'
   t = 'smart_' + ''.join(n) + '_tan'
   Test[s] = Test[i].apply(Sigmoid)
   Test[t] = Test[i].apply(TanH)

In [16]:
cf = ['smart_1_raw', 'smart_1_sig', 'smart_1_tan', 'smart_7_raw', 'smart_7_sig', 'smart_7_tan', 'smart_188_raw', 'smart_188_sig', 'smart_188_tan', 'smart_193_raw', 'smart_193_sig', 'smart_193_tan', 'smart_195_raw', 'smart_195_sig', 'smart_195_tan', 'smart_240_raw', 'smart_240_sig', 'smart_240_tan', 'smart_241_raw', 'smart_241_sig', 'smart_241_tan', 'smart_242_raw', 'smart_242_sig', 'smart_242_tan']
j = 1
for i in cf:
  print(i + ' feature' + ' Correlation with target(Failure)' + ' is ' + str(Test[i].corr(Test['failure']))) 
  if(j % 3 == 0):
    print()
  j+=1

smart_1_raw feature Correlation with target(Failure) is 0.0016745163119723456
smart_1_sig feature Correlation with target(Failure) is 0.0038934864467412077
smart_1_tan feature Correlation with target(Failure) is 0.003629195867316591

smart_7_raw feature Correlation with target(Failure) is 0.0006028239845759019
smart_7_sig feature Correlation with target(Failure) is 0.003924482014598708
smart_7_tan feature Correlation with target(Failure) is 0.0036595145913076196

smart_188_raw feature Correlation with target(Failure) is 0.009120508594145146
smart_188_sig feature Correlation with target(Failure) is 0.0009181341247338716
smart_188_tan feature Correlation with target(Failure) is 0.0009957624218700804

smart_193_raw feature Correlation with target(Failure) is -0.0014046697204491488
smart_193_sig feature Correlation with target(Failure) is 0.0006273522436118276
smart_193_tan feature Correlation with target(Failure) is 0.0004853723656640287

smart_195_raw feature Correlation with target(Fail

In [17]:
cf = ['smart_1_raw', 'smart_7_raw', 'smart_193_raw', 'smart_195_raw', 'smart_240_raw', 'smart_242_raw']
for i in cf:
   n = re.findall('\d+',i) 
   s = 'smart_' + ''.join(n) + '_sig'
   Data[s] = Data[i].apply(Sigmoid)

print("Final shape of Data : ", Data.shape)

Final shape of Data :  (553328, 57)


In [18]:
def res_fit(cat, Y):
    j = dict(cat.value_counts())  
    true, false = 0, 0
    TRUE, FALSE = {}, {} 
    for key, value in j.items(): 
        sum, neg, = 0, 0
        for state, y in zip(cat, Y): 
            if (key == state and y == 1): 
                sum+= 1             
            elif (key == state and y == 0):
                neg+= 1              
        true = sum / value           
        false = neg / value          
        TRUE[key] = true
        FALSE[key] = false           
    return j, TRUE, FALSE

def res_transform(cat, TRUE, FALSE):
    t = []
    f = []
    for state in cat: 
        for ((key_t, value_t), (key_f, value_f)) in zip(TRUE.items(), FALSE.items()):  

            if state == key_t and state == key_f: 
                t.append(value_t)                 
                f.append(value_f)
                break
        else :           
                t.append(1/2)
                f.append(1/2) 

    X_t = np.array(t).reshape(-1, 1)
    X_f = np.array(f).reshape(-1, 1) 
    
    return np.concatenate((X_t, X_f), axis = 1) 

In [19]:
del Test

In [20]:
split_date = '2019-07-03'

In [21]:
Train = Data.loc[Data.date <= split_date]
Val = Data.loc[Data.date > split_date]
print(Train.shape, Val.shape)

values, TR, FA = res_fit(Train['model'], Train['failure'].values)
print(values)
print()
Train_model = res_transform(Train['model'], TR, FA)
Val_model = res_transform(Val['model'], TR, FA)
print('*'*50)
print(Train_model.shape)
print(Val_model.shape)

(331926, 57) (221402, 57)
{'ST12000NM0007': 104129, 'ST4000DM000': 58705, 'ST8000NM0055': 43142, 'HGST HMS5C4040BLE640': 38256, 'ST8000DM002': 29622, 'HGST HUH721212ALN604': 28835, 'HGST HMS5C4040ALE640': 7921, 'TOSHIBA MG07ACA14TA': 3660, 'ST10000NM0086': 3600, 'HGST HUH728080ALE600': 3003, 'ST6000DX000': 2658, 'HGST HUH721212ALE600': 1560, 'ST500LM012 HN': 1509, 'TOSHIBA MQ01ABF050': 1497, 'TOSHIBA MQ01ABF050M': 1134, 'ST500LM030': 699, 'WDC WD5000LPVX': 636, 'TOSHIBA MD04ABA400V': 297, 'WDC WD5000LPCX': 162, 'ST4000DM005': 120, 'Seagate SSD': 111, 'ST500LM021': 97, 'HGST HUS726040ALE610': 84, 'HGST HDS5C4040ALE630': 78, 'ST8000DM005': 75, 'TOSHIBA HDWF180': 60, 'DELLBOSS VD': 60, 'HGST HUH721010ALE600': 60, 'Seagate BarraCuda SSD ZA500CM10002': 54, 'WDC WD5000BPKT': 33, 'ST6000DM001': 12, 'TOSHIBA HDWE160': 12, 'Seagate BarraCuda SSD ZA2000CM10002': 9, 'WDC WD60EFRX': 9, 'ST8000DM004': 9, 'Hitachi HDS5C4040ALE630': 3, 'ST320LT007': 3, 'ST9250315AS': 3, 'HGST HMS5C4040BLE641': 3, 'ST

In [22]:
X_Train_orig = Train.drop(f, axis = 1).values
Y_Train_orig = Train.failure.values
X_Val_orig = Val.drop(f, axis = 1).values
Y_Val_orig = Val.failure.values
X_Train_orig.shape, Y_Train_orig.shape
X_Val_orig.shape, Y_Val_orig.shape

((221402, 52), (221402,))

In [23]:
X_Train_orig = np.hstack((X_Train_orig, Train_model))
X_Val_orig = np.hstack((X_Val_orig, Val_model))
X_Train_orig.shape, X_Val_orig.shape

((331926, 54), (221402, 54))

In [24]:
del Train_model
del Val_model

In [25]:
Counter(Y_Train_orig), Counter(Y_Val_orig)

(Counter({0: 331906, 1: 20}), Counter({0: 221397, 1: 5}))

In [26]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

under = RandomUnderSampler(sampling_strategy = 0.8)
over_smote = SMOTE(n_jobs = -1, k_neighbors = 1, sampling_strategy = 0.5)
steps = [('o', over_smote), ('u', under)]
pipeline = Pipeline(steps = steps)

In [27]:
x_train_sam, y_train_sam = pipeline.fit_resample(X_Train_orig, Y_Train_orig)
x_val, y_val = X_Val_orig, Y_Val_orig
x_train_sam.shape, y_train_sam.shape 
x_val.shape, y_val.shape 

from sklearn.preprocessing import Normalizer, StandardScaler
normalizer = Normalizer()
sc = StandardScaler()

sc.fit(x_train_sam)

X_train_standard = sc.transform(x_train_sam)
X_val_standard = sc.transform(x_val)

C:\Users\India\anaconda3\lib\site-packages\imblearn\over_sampling\_smote\base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


In [ ]:
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify, render_template
import pickle
import os
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import SMOTE
from math import log

app = Flask(__name__)

model = pickle.load(open('model.pkl', 'rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    try:
        file=request.files['file']          
        dat = pd.read_csv(file)  
        dat = dat.append(pd.read_csv("C:/Users/India/Documents/Project2k23/Hard Disk failure/dataset/2019-07-13.csv"))
        dat.reset_index(inplace = True)
        dat.drop('index', axis = 1, inplace = True)
        dat.reset_index(inplace = True)
        dat.drop('index', axis = 1, inplace = True)
        dat.drop(Initial[Null], axis = 1, inplace = True)
        dat.drop(M_features[index], axis = 1, inplace = True)
        dat.drop(np.where(dat['capacity_bytes'] == -1)[0], inplace = True)
        dat = dat.fillna(0)

        for i in cf:
           n = re.findall('\d+',i) 
           s = 'smart_' + ''.join(n) + '_sig'
           dat[s] = dat[i].apply(Sigmoid)
        print(dat.shape)
        values, TR, FA = res_fit(dat['model'], dat['failure'].values)
        print(values)
        Test_model = res_transform(dat['model'], TR, FA)
        X_Test_orig = dat.drop(f, axis = 1).values
        Y_Test_orig = dat.failure.values

        X_Test_orig = np.hstack((X_Test_orig, Test_model))
        X_Test_orig.shape

        X_test, y_test = X_Test_orig, Y_Test_orig

        
        y_test.shape
        Y_predicted_test = model.predict(X_test)
        if Counter(y_test)[1] < 5:
            pred = '95% of Hard Disk Failure'
        elif Counter(y_test)[1] < 10:
            pred ='90% of Hard Disk Failure'
        elif Counter(y_test)[1] < 15:
            pred ='85% of Hard Disk Failure'
        elif Counter(y_test)[1] < 20:
            pred ='80% of Hard Disk Failure'
        elif Counter(y_test)[1] < 25:
            pred ='75% of Hard Disk Failure'
        else:
            pred ='Below 50% of Hard Disk Failure'
        
        output = pred
        return render_template('index.html', prediction_text='{}'.format(output))

    except:
        flash("Chech Your Name And Password")   


if __name__ == "__main__":
    app.run(host='0.0.0.0', port='600')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:600
 * Running on http://192.168.1.32:600
Press CTRL+C to quit
127.0.0.1 - - [19/Apr/2023 16:23:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 16:23:47] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 16:23:47] "GET /static/bg.jpg HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 16:23:47] "GET /favicon.ico HTTP/1.1" 404 -
C:\Users\India\AppData\Local\Temp\ipykernel_19308\1183530707.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dat = dat.append(pd.read_csv("C:/Users/India/Documents/Project2k23/Hard Disk failure/dataset/2019-07-13.csv"))


(221986, 57)
{'ST12000NM0007': 69420, 'ST4000DM000': 39160, 'ST8000NM0055': 28771, 'HGST HMS5C4040BLE640': 25504, 'ST8000DM002': 19740, 'HGST HUH721212ALN604': 19336, 'HGST HMS5C4040ALE640': 5303, 'TOSHIBA MG07ACA14TA': 2440, 'ST10000NM0086': 2400, 'HGST HUH728080ALE600': 2002, 'ST6000DX000': 1772, 'HGST HUH721212ALE600': 1560, 'ST500LM012 HN': 1006, 'TOSHIBA MQ01ABF050': 994, 'TOSHIBA MQ01ABF050M': 758, 'ST500LM030': 466, 'WDC WD5000LPVX': 427, 'TOSHIBA MD04ABA400V': 198, 'WDC WD5000LPCX': 108, 'ST4000DM005': 80, 'Seagate SSD': 74, 'ST500LM021': 66, 'DELLBOSS VD': 60, 'HGST HUS726040ALE610': 56, 'HGST HDS5C4040ALE630': 52, 'ST8000DM005': 50, 'TOSHIBA HDWF180': 40, 'HGST HUH721010ALE600': 40, 'Seagate BarraCuda SSD ZA500CM10002': 35, 'WDC WD5000BPKT': 22, 'ST6000DM001': 8, 'TOSHIBA HDWE160': 8, 'Seagate BarraCuda SSD ZA2000CM10002': 6, 'WDC WD60EFRX': 6, 'ST8000DM004': 6, 'Hitachi HDS5C4040ALE630': 2, 'ST320LT007': 2, 'ST9250315AS': 2, 'HGST HMS5C4040BLE641': 2, 'ST1000LM024 HN': 2, 'S

127.0.0.1 - - [19/Apr/2023 16:24:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 16:24:28] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Apr/2023 16:24:28] "GET /static/bg.jpg HTTP/1.1" 304 -
